# Recurrent Neural Network

### Imports

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# for reproducibility
torch.manual_seed(100)

### Dataset Construction

In [2]:
# Dictionary
sample_sentence = 'hi!hello.'
char_set = list(set(sample_sentence))
dic = {c: i for i, c in enumerate(char_set)}

# Parameters
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size

# Dataset setting
x_batch = []
y_batch = []

x_data = [dic[c] for c in sample_sentence[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [dic[c] for c in sample_sentence[1:]]

x_batch.append(x_one_hot)
y_batch.append(y_data)

# To torch tensors
X = torch.FloatTensor(x_batch)
Y = torch.LongTensor(y_batch)
print(X.shape)
print(Y.shape)

torch.Size([1, 8, 7])
torch.Size([1, 8])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


### 2. RNN model

- Input(입력의 형태)
  + Input type: torch.Tensor
  + Input shape: (N x S x E)
    * N: Batch size, S: Sequence length, E: Embedding size
    * 단, <b>batch_first=True</b>일 때
  + 입력값 'hi!hello'
    * (1, 8, 7)

- Hidden(출력의 형태)
  + Hidden type: torch.Tensor
  + 출력값 'i!hello'
    * (1, 8, 7)
    

In [3]:
# Model
learning_rate = 0.1
training_epochs = 50
model = nn.RNN(input_size, hidden_size, batch_first=True)

In [4]:
# define cost/loss & optimizer
criterion = nn.CrossEntropyLoss()    # Softmax
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# train
for epoch in range(training_epochs):
  optimizer.zero_grad()
  outputs, _status = model(X)
  loss = criterion(outputs.reshape(-1, dic_size), Y.reshape(-1))
  loss.backward()
  optimizer.step()
  if epoch % 5 == 4:
    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print('epoch: ',epoch, 'loss: ', loss.item(), 'prediction: ', result_str, 'true Y: ', sample_sentence[1:])

epoch:  4 loss:  1.3552616834640503 prediction:  i!heel.. true Y:  i!hello.
epoch:  9 loss:  0.9755570888519287 prediction:  i!hehlo. true Y:  i!hello.
epoch:  14 loss:  0.7983919382095337 prediction:  i!hello. true Y:  i!hello.
epoch:  19 loss:  0.7118602991104126 prediction:  i!hello. true Y:  i!hello.
epoch:  24 loss:  0.6624301671981812 prediction:  i!hello. true Y:  i!hello.
epoch:  29 loss:  0.641640841960907 prediction:  i!hello. true Y:  i!hello.
epoch:  34 loss:  0.6319407820701599 prediction:  i!hello. true Y:  i!hello.
epoch:  39 loss:  0.6269137263298035 prediction:  i!hello. true Y:  i!hello.
epoch:  44 loss:  0.6242977380752563 prediction:  i!hello. true Y:  i!hello.
epoch:  49 loss:  0.6222154498100281 prediction:  i!hello. true Y:  i!hello.


### 3. Assignment

다음 3개의 문장을 batch data로 활용해 RNN을 학습해보자 (아래의 미완성 코드, 위 실습코드, 실행결과를 참고)

- 'howareyou'
- 'whats up?'
- 'iamgreat'

In [5]:
# import
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# for reproducibility
torch.manual_seed(100)

# Dictionary. 샘플 문장에 쓰인 글자 수가 모두 9로 같다.
sample_sentences = ['howareyou', 'whats up?', 'iamgreat.']
char_set = list(set(''.join(sample_sentences)))
dic = {c: i for i, c in enumerate(char_set)}

print(sample_sentences[0])
print(len(sample_sentences[0]))
print(sample_sentences[1])
print(len(sample_sentences[1]))
print(sample_sentences[2])
print(len(sample_sentences[2]))

# 샘플 문장의 수는 세개, 16개의 문자가 char_set에 입력되며, 그 중에는 ' '도 포함된다.
print(sample_sentences)
print(char_set)
print(dic)

# Parameters. Input으로 17개의 글자가 들어감으로 17개 ,hidden size = Output Size이기 때문에 글자의 갯수인 17개가 맞다.
# 결국 sample_sentences의 문자들이 인풋과 아웃풋의 후보가 되므로, sample_sentences에 쓰인 문자의 갯수인 len(dic)이 결국 인풋과 아웃풋의 사이즈가 되는 것이다.
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size

# 딕셔너리 형으로 정의된 dic의 크기. dic은 char_set을 key로 사용하고, 숫자는 char_set의 순서에 따라 매긴 것이므로 dic과 char_set의 크기는 같다.
print(dic_size)
print(len(char_set))

# Dataset setting. 인풋 batch, 타겟 batch 
input_batch = []
target_batch = []

# 각 sentence를 학습시킬 때 for문을 사용하여 순차적으로 howareyou부터 iamgreat까지 순차적으로 학습한다.
# 학습시킬 sentence의 차례에는 x_data에 sentence, x_one_hot에 원핫코드, y_data에 정답을 입력한다.
# 따라서 모델의 input으로 들어갈 수 있는 글자는 sentence에 포함되어있는 글자이다.
# 또한 모델의 output으로 나올 수 있는 글자도 sentence에 포함되어있는 글자가 되며 이는 sentence의 두번째 글자 부터이다.
for sentence in sample_sentences:
  x_data = [dic[c] for c in sample_sentences[sample_sentences.index(sentence)][:-1]]
  x_one_hot = [np.eye(dic_size)[x] for x in x_data]
  y_data = [dic[c] for c in sample_sentences[sample_sentences.index(sentence)][1:]]

# x_one_hot, y_data를 batch(List형)에 넣은 후 Tensor로 변환하는 과정. 모델의 학습은 Tensor를 통해 이루어진다.
  input_batch.append(x_one_hot)
  target_batch.append(y_data)

  IpTs = torch.FloatTensor(input_batch)
  tgTs = torch.LongTensor(target_batch)

  # Model. 강의 자료에 있는 값이 0.05, 500으로 고정. 더 원활한 학습을 위해서는 파라미터의 수정이 필요하다.
  learning_rate = 0.05
  training_epochs = 500
  model = nn.RNN(input_size, hidden_size, batch_first=True)

  # define cost/loss & optimizer (분류 과정을 거치므로 CrossEntropyLoss 함수의 도움을 받을 수 있다. Adam은 SGD방식과 유사하지만 더 진화한 알고리즘을 가진다.)
  criterion = nn.CrossEntropyLoss()    # Softmax
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  # train
  for epoch in range(training_epochs):
    optimizer.zero_grad()
    outputs, _status = model(IpTs)
    loss = criterion(outputs.reshape(-1, dic_size), tgTs.reshape(-1))
    loss.backward()
    optimizer.step()
    if epoch %100 == 99:
      result = outputs.data.numpy().argmax(axis=2)
for sentence in result:
        print(''.join([char_set[c] for c in np.squeeze(sentence)]))

howareyou
9
whats up?
9
iamgreat.
9
['howareyou', 'whats up?', 'iamgreat.']
['?', 't', 'm', 'w', 'p', 'e', 'u', 's', 'i', '.', 'y', 'r', 'a', ' ', 'h', 'g', 'o']
{'?': 0, 't': 1, 'm': 2, 'w': 3, 'p': 4, 'e': 5, 'u': 6, 's': 7, 'i': 8, '.': 9, 'y': 10, 'r': 11, 'a': 12, ' ': 13, 'h': 14, 'g': 15, 'o': 16}
17
17
owareaou
hats up?
amgreat.
